## You can also run the notebook in [COLAB](https://colab.research.google.com/github/deepmipt/DeepPavlov/blob/master/examples/gobot_tutorial.ipynb).

In [ ]:
!pip install deeppavlov

## Data Preparation

In [3]:
from deeppavlov.dataset_readers.dstc2_reader import DSTC2DatasetReader

data = DSTC2DatasetReader().read('my_data')

2019-08-09 18:04:37.130 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 95: [downloading data from http://files.deeppavlov.ai/datasets/dstc2_v2.tar.gz to my_data]
2019-08-09 18:04:37.137 INFO in 'deeppavlov.core.data.utils'['utils'] at line 63: Downloading from http://files.deeppavlov.ai/datasets/dstc2_v2.tar.gz to my_data/dstc2_v2.tar.gz
100%|██████████| 506k/506k [00:00<00:00, 37.1MB/s]
2019-08-09 18:04:37.155 INFO in 'deeppavlov.core.data.utils'['utils'] at line 201: Extracting my_data/dstc2_v2.tar.gz archive into my_data
2019-08-09 18:04:37.177 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from my_data/dstc2-trn.jsonlist]
2019-08-09 18:04:37.346 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from my_data/dstc2-val.jsonlist]
2019-08-09 18:04:37.493 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from my_data/dstc

In [4]:
data['train'][1]

({'text': 'cheap restaurant',
  'intents': [{'slots': [['pricerange', 'cheap']], 'act': 'inform'}]},
 {'text': 'What kind of food would you like?', 'act': 'request_food'})

In [5]:
from deeppavlov.dataset_iterators.dialog_iterator import DialogDatasetIterator

iterator = DialogDatasetIterator(data)

In [6]:
x_dialog, y_dialog = iterator.train[0]
x_dialog

[{'text': '', 'intents': [], 'prev_resp_act': None},
 {'text': 'cheap restaurant',
  'intents': [{'slots': [['pricerange', 'cheap']], 'act': 'inform'}],
  'prev_resp_act': 'welcomemsg'},
 {'text': 'any',
  'intents': [{'slots': [['this', 'dontcare']], 'act': 'inform'}],
  'prev_resp_act': 'request_food'},
 {'text': 'south',
  'intents': [{'slots': [['area', 'south']], 'act': 'inform'}],
  'prev_resp_act': 'request_area'},
 {'text': 'south',
  'intents': [{'slots': [['area', 'south']], 'act': 'inform'}],
  'db_result': {'food': 'chinese',
   'pricerange': 'cheap',
   'area': 'south',
   'addr': 'cambridge leisure park clifton way cherry hinton',
   'phone': '01223 244277',
   'postcode': 'c.b 1, 7 d.y',
   'name': 'the lucky star'},
  'prev_resp_act': 'api_call'},
 {'text': 'address',
  'intents': [{'slots': [['slot', 'addr']], 'act': 'request'}],
  'prev_resp_act': 'inform_area+inform_food+offer_name'},
 {'text': 'phone number',
  'intents': [{'slots': [['slot', 'phone']], 'act': 'requ

## 1. Build database of items

In [7]:
from deeppavlov.core.data.sqlite_database import Sqlite3Database

database = Sqlite3Database(table_name="mytable",
                           primary_keys=["name"],
                           keys=["name", "food", "pricerange", "area", "addr", "phone", "postcode"],
                           save_path="my_bot/db.sqlite")

2019-08-09 18:04:39.554 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 47: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2019-08-09 18:04:39.555 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 67: Initializing empty database on /home/vimary/code-projects/Pilot/examples/my_bot/db.sqlite.


In [8]:
db_results = []
for dialog in iterator.gen_batches(batch_size=1, data_type='all'):
    turns_x, turns_y = dialog
    db_results.extend(x['db_result'] for x in turns_x[0] if x.get('db_result'))

print(f"Adding {len(db_results)} items.")
if db_results:
    database.fit(db_results)

2019-08-09 18:04:40.105 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 138: Created table with keys {'name': 'text', 'food': 'text', 'pricerange': 'text', 'area': 'text', 'addr': 'text', 'phone': 'text', 'postcode': 'text'}.


Adding 3016 items.


In [9]:
database([{'pricerange': 'cheap', 'area': 'south'}])

[[{'name': 'the lucky star',
   'food': 'chinese',
   'pricerange': 'cheap',
   'area': 'south',
   'addr': 'cambridge leisure park clifton way cherry hinton',
   'phone': '01223 244277',
   'postcode': 'c.b 1, 7 d.y'},
  {'name': 'nandos',
   'food': 'portuguese',
   'pricerange': 'cheap',
   'area': 'south',
   'addr': 'cambridge leisure park clifton way',
   'phone': '01223 327908',
   'postcode': 'c.b 1, 7 d.y'}]]

In [10]:
!ls my_bot

db.sqlite


## 2. Build Slot Filler

Slot Filler is component that inputs text and outputs dictionary of slot names and their values:

    slot_filler(['I would like some chineese food'])
    >> [{'food': 'chinese'}]

To implement a slot filler you need to provide
    
 - **slot types**
 - all possible **slot values**
 - optionally, it will be good to provide examples of mentions for every value of a slot
 
The data should be provided in `slot_vals.json` file with the following format:

    {
        'food': {
            'chinese': ['chinese', 'chineese', 'chines'],
            'french': ['french', 'freench'],
            'dontcare': ['any food', 'any type of food']
        }
    }
                

There are two possible models for a slot filler:

   1. A non-trainable levenshtein distance based slot filler [[tutor section]](#2.1.-Non-trainable-slot-filler)
   2. A tranaible slot filler with a neural network solving the task of named entity tagging [[tutor section]](#2.2.-Trainable-slot-filler)

In [11]:
from deeppavlov.download import download_decompress

download_decompress(url='http://files.deeppavlov.ai/deeppavlov_data/dstc_slot_vals.tar.gz',
                    download_path='my_bot/slotfill')

2019-08-09 18:04:42.256 INFO in 'deeppavlov.core.data.utils'['utils'] at line 63: Downloading from http://files.deeppavlov.ai/deeppavlov_data/dstc_slot_vals.tar.gz to my_bot/slotfill/dstc_slot_vals.tar.gz
100%|██████████| 1.62k/1.62k [00:00<00:00, 5.77MB/s]
2019-08-09 18:04:42.258 INFO in 'deeppavlov.core.data.utils'['utils'] at line 201: Extracting my_bot/slotfill/dstc_slot_vals.tar.gz archive into my_bot/slotfill


In [12]:
!ls my_bot/slotfill

dstc_slot_vals.json


### 2.1. Non-trainable slot filler

In [13]:
from deeppavlov import configs
from deeppavlov.core.common.file import read_json


slotfill_raw_config = read_json(configs.ner.slotfill_dstc2_raw)

In [14]:
slotfill_raw_config['metadata']['variables']['DATA_PATH'] = 'my_data'
slotfill_raw_config['metadata']['variables']['SLOT_VALS_PATH'] = 'my_bot/slotfill/dstc_slot_vals.json'

In [15]:
from deeppavlov import evaluate_model

slotfill_raw = evaluate_model(slotfill_raw_config);

2019-08-09 18:04:44.668 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-trn.jsonlist]
2019-08-09 18:04:44.851 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-val.jsonlist]
2019-08-09 18:04:44.999 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-tst.jsonlist]
[nltk_data] Downloading package punkt to /home/vimary/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/vimary/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/vimary/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading

{"valid": {"eval_examples_count": 1444, "metrics": {"slots_accuracy": 0.9307}, "time_spent": "0:00:27"}}
{"test": {"eval_examples_count": 1386, "metrics": {"slots_accuracy": 0.9481}, "time_spent": "0:00:25"}}


We've got slot accuracy of 93% on valid set and 94% on test set.

In [16]:
from deeppavlov import build_model

slotfill_raw = build_model(slotfill_raw_config)

In [17]:
slotfill_raw(['i want cheap chinese food'])

[{'food': 'chinese', 'pricerange': 'cheap'}]

### 2.2. Trainable slot filler

In [18]:
from deeppavlov import configs
from deeppavlov.core.common.file import read_json


ner_config = read_json(configs.ner.ner_dstc2)

In [19]:
ner_config['metadata']['variables']['DATA_PATH'] = 'my_data'
ner_config['metadata']['variables']['SLOT_VALS_PATH'] = 'my_bot/slotfill/dstc_slot_vals.json'
ner_config['metadata']['variables']['MODEL_PATH'] = 'my_bot/slotfill'

In [20]:
from deeppavlov import train_model

train_model(ner_config);

2019-08-09 18:06:19.6 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-trn.jsonlist]
2019-08-09 18:06:19.241 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-val.jsonlist]
2019-08-09 18:06:19.440 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-tst.jsonlist]
2019-08-09 18:06:19.748 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 98: [saving vocabulary to /home/vimary/code-projects/Pilot/examples/my_bot/slotfill/word.dict]
2019-08-09 18:06:19.757 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 98: [saving vocabulary to /home/vimary/code-projects/Pilot/examples/my_bot/slotfill/tag.dict]
W0809 18:06:20.605350 139777813403264 deprec

{"valid": {"eval_examples_count": 1444, "metrics": {"ner_f1": 10.1133, "per_token_accuracy": 0.0715}, "time_spent": "0:00:01", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 5}}


2019-08-09 18:06:23.849 DEBUG in 'deeppavlov.metrics.fmeasure'['fmeasure'] at line 390: processed 422 tokens with 62 phrases; found: 62 phrases; correct: 0.

precision:  98.39%; recall:  98.39%; FB1:  98.39

	area: precision:  95.45%; recall:  100.00%; F1:  97.67 22

	food: precision:  100.00%; recall:  100.00%; F1:  100.00 25

	name: precision:  0.00%; recall:  0.00%; F1:  0.00 0

	pricerange: precision:  100.00%; recall:  100.00%; F1:  100.00 15


I0809 18:06:23.849567 139777813403264 fmeasure.py:390] processed 422 tokens with 62 phrases; found: 62 phrases; correct: 0.

precision:  98.39%; recall:  98.39%; FB1:  98.39

	area: precision:  95.45%; recall:  100.00%; F1:  97.67 22

	food: precision:  100.00%; recall:  100.00%; F1:  100.00 25

	name: precision:  0.00%; recall:  0.00%; F1:  0.00 0

	pricerange: precision:  100.00%; recall:  100.00%; F1:  100.00 15




{"train": {"eval_examples_count": 64, "metrics": {"ner_f1": 98.3871, "per_token_accuracy": 0.9953}, "time_spent": "0:00:02", "epochs_done": 3, "batches_seen": 100, "train_examples_seen": 6247, "learning_rate": 0.01, "momentum": null, "loss": 0.07777087733149529}}


2019-08-09 18:06:24.220 DEBUG in 'deeppavlov.metrics.fmeasure'['fmeasure'] at line 390: processed 8424 tokens with 1244 phrases; found: 1258 phrases; correct: 0.

precision:  96.26%; recall:  97.35%; FB1:  96.80

	area: precision:  98.34%; recall:  99.00%; F1:  98.67 302

	food: precision:  96.65%; recall:  97.58%; F1:  97.11 626

	name: precision:  0.00%; recall:  0.00%; F1:  0.00 0

	pricerange: precision:  96.94%; recall:  98.28%; F1:  97.60 294

	this: precision:  66.67%; recall:  75.00%; F1:  70.59 36


I0809 18:06:24.220485 139777813403264 fmeasure.py:390] processed 8424 tokens with 1244 phrases; found: 1258 phrases; correct: 0.

precision:  96.26%; recall:  97.35%; FB1:  96.80

	area: precision:  98.34%; recall:  99.00%; F1:  98.67 302

	food: precision:  96.65%; recall:  97.58%; F1:  97.11 626

	name: precision:  0.00%; recall:  0.00%; F1:  0.00 0

	pricerange: precision:  96.94%; recall:  98.28%; F1:  97.60 294

	this: precision:  66.67%; recall:  75.00%; F1:  70.59 36


2019-

...

2019-08-09 18:06:39.308 DEBUG in 'deeppavlov.metrics.fmeasure'['fmeasure'] at line 390: processed 8424 tokens with 1244 phrases; found: 1226 phrases; correct: 0.

precision:  98.86%; recall:  97.43%; FB1:  98.14

	area: precision:  99.67%; recall:  99.67%; F1:  99.67 300

	food: precision:  99.34%; recall:  96.61%; F1:  97.96 603

	name: precision:  0.00%; recall:  0.00%; F1:  0.00 2

	pricerange: precision:  98.29%; recall:  98.97%; F1:  98.63 292

	this: precision:  93.10%; recall:  84.38%; F1:  88.52 29


I0809 18:06:39.308403 139777813403264 fmeasure.py:390] processed 8424 tokens with 1244 phrases; found: 1226 phrases; correct: 0.

precision:  98.86%; recall:  97.43%; FB1:  98.14

	area: precision:  99.67%; recall:  99.67%; F1:  99.67 300

	food: precision:  99.34%; recall:  96.61%; F1:  97.96 603

	name: precision:  0.00%; recall:  0.00%; F1:  0.00 2

	pricerange: precision:  98.29%; recall:  98.97%; F1:  98.63 292

	this: precision:  93.10%; recall:  84.38%; F1:  88.52 29


2019-

{"valid": {"eval_examples_count": 1444, "metrics": {"ner_f1": 98.1377, "per_token_accuracy": 0.9932}, "time_spent": "0:00:18", "epochs_done": 80, "batches_seen": 2480, "train_examples_seen": 154640, "impatience": 5, "patience_limit": 5}}


2019-08-09 18:06:39.986 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 52: [loading model from /home/vimary/code-projects/Pilot/examples/my_bot/slotfill/model]
I0809 18:06:39.986219 139777813403264 tf_model.py:52] [loading model from /home/vimary/code-projects/Pilot/examples/my_bot/slotfill/model]
2019-08-09 18:06:40.219 DEBUG in 'deeppavlov.metrics.fmeasure'['fmeasure'] at line 390: processed 8424 tokens with 1244 phrases; found: 1232 phrases; correct: 0.

precision:  98.86%; recall:  97.91%; FB1:  98.38

	area: precision:  99.67%; recall:  99.67%; F1:  99.67 300

	food: precision:  99.18%; recall:  97.26%; F1:  98.21 608

	name: precision:  0.00%; recall:  0.00%; F1:  0.00 1

	pricerange: precision:  98.29%; recall:  98.97%; F1:  98.63 292

	this: precision:  93.55%; recall:  90.62%; F1:  92.06 31


I0809 18:06:40.219339 139777813403264 fmeasure.py:390] processed 8424 tokens with 1244 phrases; found: 1232 phrases; correct: 0.

precision:  98.86%; recall:  97.91%; FB1: 

{"valid": {"eval_examples_count": 1444, "metrics": {"ner_f1": 98.3845, "per_token_accuracy": 0.9941}, "time_spent": "0:00:01"}}
{"test": {"eval_examples_count": 1386, "metrics": {"ner_f1": 97.1742, "per_token_accuracy": 0.9904}, "time_spent": "0:00:01"}}


2019-08-09 18:06:40.978 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 52: [loading model from /home/vimary/code-projects/Pilot/examples/my_bot/slotfill/model]
I0809 18:06:40.978024 139777813403264 tf_model.py:52] [loading model from /home/vimary/code-projects/Pilot/examples/my_bot/slotfill/model]


Tags (& counts) found in the data are:

In [21]:
!cat my_bot/slotfill/tag.dict

O	9973
B-food	777
B-area	491
B-pricerange	408
I-food	41
I-pricerange	40
B-this	36
I-this	33
I-area	26
B-name	8
I-name	8


In [22]:
from deeppavlov import evaluate_model

evaluate_model(ner_config);

2019-08-09 18:06:41.206 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-trn.jsonlist]
I0809 18:06:41.206093 139777813403264 dstc2_reader.py:112] [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-trn.jsonlist]
2019-08-09 18:06:41.599 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-val.jsonlist]
I0809 18:06:41.599298 139777813403264 dstc2_reader.py:112] [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-val.jsonlist]
2019-08-09 18:06:41.868 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-tst.jsonlist]
I0809 18:06:41.868273 139777813403264 dstc2_reader.py:112] [loading dialogs from /home/vimary/code-projects/Pilot

{"valid": {"eval_examples_count": 1444, "metrics": {"ner_f1": 98.3845, "per_token_accuracy": 0.9941}, "time_spent": "0:00:01"}}
{"test": {"eval_examples_count": 1386, "metrics": {"ner_f1": 97.1742, "per_token_accuracy": 0.9904}, "time_spent": "0:00:01"}}


We've got tagging F1-score=98 on valid set and 97 on test set.

In [23]:
from deeppavlov import build_model

ner = build_model(ner_config)

2019-08-09 18:07:35.901 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 112: [loading vocabulary from /home/vimary/code-projects/Pilot/examples/my_bot/slotfill/word.dict]
I0809 18:07:35.901416 139777813403264 simple_vocab.py:112] [loading vocabulary from /home/vimary/code-projects/Pilot/examples/my_bot/slotfill/word.dict]
2019-08-09 18:07:35.904 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 112: [loading vocabulary from /home/vimary/code-projects/Pilot/examples/my_bot/slotfill/tag.dict]
I0809 18:07:35.904371 139777813403264 simple_vocab.py:112] [loading vocabulary from /home/vimary/code-projects/Pilot/examples/my_bot/slotfill/tag.dict]
2019-08-09 18:07:36.454 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 52: [loading model from /home/vimary/code-projects/Pilot/examples/my_bot/slotfill/model]
I0809 18:07:36.454594 139777813403264 tf_model.py:52] [loading model from /home/vimary/code-projects/Pilot/examples/my_bot/slotfill/model]


In [24]:
ner(['hi i want some cheap chinese food'])

[[['hi', 'i', 'want', 'some', 'cheap', 'chinese', 'food']],
 [['O', 'O', 'O', 'O', 'B-pricerange', 'B-food', 'O']]]

In [25]:
import json

json.dump(ner_config, open('my_bot/ner_config.json', 'wt'), indent=2)

!ls my_bot

db.sqlite  ner_config.json  slotfill


In [26]:
slotfill_config = read_json(configs.ner.slotfill_dstc2)

In [27]:
slotfill_config['metadata']['variables']['DATA_PATH'] = 'my_data'
slotfill_config['metadata']['variables']['SLOT_VALS_PATH'] = 'my_bot/slotfill/dstc_slot_vals.json'

slotfill_config['metadata']['variables']['MODEL_PATH'] = 'my_bot/slotfill'
slotfill_config['metadata']['variables']['NER_CONFIG_PATH'] = 'my_bot/ner_config.json'

In [28]:
evaluate_model(slotfill_config);

2019-08-09 18:07:43.486 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-trn.jsonlist]
I0809 18:07:43.486081 139777813403264 dstc2_reader.py:112] [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-trn.jsonlist]
2019-08-09 18:07:43.598 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-val.jsonlist]
I0809 18:07:43.598820 139777813403264 dstc2_reader.py:112] [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-val.jsonlist]
2019-08-09 18:07:43.902 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-tst.jsonlist]
I0809 18:07:43.902823 139777813403264 dstc2_reader.py:112] [loading dialogs from /home/vimary/code-projects/Pilot

{"valid": {"eval_examples_count": 1444, "metrics": {"slots_accuracy": 0.9778}, "time_spent": "0:00:11"}}
{"test": {"eval_examples_count": 1386, "metrics": {"slots_accuracy": 0.9683}, "time_spent": "0:00:10"}}


We've got slot accuracy of 97% on valid set and 96% on test set.

In [29]:
slotfill = build_model(slotfill_config)

2019-08-09 18:08:14.305 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 112: [loading vocabulary from /home/vimary/code-projects/Pilot/examples/my_bot/slotfill/word.dict]
I0809 18:08:14.305612 139777813403264 simple_vocab.py:112] [loading vocabulary from /home/vimary/code-projects/Pilot/examples/my_bot/slotfill/word.dict]
2019-08-09 18:08:14.308 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 112: [loading vocabulary from /home/vimary/code-projects/Pilot/examples/my_bot/slotfill/tag.dict]
I0809 18:08:14.308433 139777813403264 simple_vocab.py:112] [loading vocabulary from /home/vimary/code-projects/Pilot/examples/my_bot/slotfill/tag.dict]
2019-08-09 18:08:14.850 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 52: [loading model from /home/vimary/code-projects/Pilot/examples/my_bot/slotfill/model]
I0809 18:08:14.850203 139777813403264 tf_model.py:52] [loading model from /home/vimary/code-projects/Pilot/examples/my_bot/slotfill/model]


In [30]:
slotfill(['i want cheap chinese food, pls'])

[{'pricerange': 'cheap', 'food': 'chinese'}]

## Train bot

In [ ]:
config = {
  "dataset_reader": {
    "class_name": "dstc2_reader",
    "data_path": "{DOWNLOADS_PATH}"
  },
  "dataset_iterator": {
    "class_name": "dialog_iterator"
  },
  "chainer": {
    "in": ["x"],
    "in_y": ["y"],
    "out": ["y_predicted"],
    "pipe": [
      {
        "class_name": "deeppavlov.models.go_bot.wrapper:DialogComponentWrapper",
        "component": {
            "class_name": "split_tokenizer"
        },
        "in": ["x"],
        "out": ["x_tokens"]
      },
      {
        "id": "word_vocab",
        "class_name": "simple_vocab",
        "fit_on": ["x_tokens"],
        "save_path": "{BOT_PATH}/word.dict",
        "load_path": "{BOT_PATH}/word.dict"
      },
      {
        "id": "restaurant_database",
        "class_name": "sqlite_database",
        "table_name": "mytable",
        "primary_keys": ["name"],
        "save_path": "{DOWNLOADS_PATH}/db.sqlite"
      },
      {
        "class_name": "go_bot",
        "load_path": "{BOT_PATH}/model",
        "save_path": "{BOT_PATH}/model",
        "in": ["x"],
        "in_y": ["y"],
        "out": ["y_predicted"],
        "main": True,
        "debug": False,
        "learning_rate": 0.003,
        "learning_rate_drop_patience": 5,
        "learning_rate_drop_div": 10.0,
        "momentum": 0.95,
        "optimizer": "tensorflow.train:AdamOptimizer",
        "clip_norm": 2.0,
        "dropout_rate": 0.4,
        "l2_reg_coef": 3e-4,
        "hidden_size": 128,
        "dense_size": 160,
        "word_vocab": "#word_vocab",
        "template_path": "{DOWNLOADS_PATH}/templates.txt",
        "template_type": "DualTemplate",
        "database": "#restaurant_database",
        "api_call_action": "api_call",
        "use_action_mask": False,
        "slot_filler": {
          "config_path": "{CONFIGS_PATH}/ner/slotfill_dstc2.json"
        },
        "intent_classifier": None,
        "embedder": {
          "class_name": "glove",
          "load_path": "{DOWNLOADS_PATH}/glove.6B.100d.txt"
        },
        "bow_embedder": {
          "class_name": "bow",
          "depth": "#word_vocab.__len__()",
          "with_counts": True
        },
        "tokenizer": {
          "class_name": "stream_spacy_tokenizer",
          "lowercase": False
        },
        "tracker": {
          "class_name": "featurized_tracker",
          "slot_names": ["pricerange", "this", "area", "food", "name"]
        }
      }
    ]
  },
  "train": {
    "epochs": 200,
    "batch_size": 8,

    "metrics": ["per_item_dialog_accuracy"],
    "validation_patience": 10,
    "val_every_n_batches": 15,

    "log_every_n_batches": 15,
    "show_examples": False,
    "evaluation_targets": [
      "valid",
      "test"
    ],
    "class_name": "nn_trainer"
  },
  "metadata": {
    "variables": {
      "ROOT_PATH": ".",
      "DOWNLOADS_PATH": "{ROOT_PATH}/my_data",
      "MODEL_PATH": "{ROOT_PATH}/my_model",
      "CONFIGS_PATH": "{DEEPPAVLOV_PATH}/configs"
    },
    "requirements": [
      "{DEEPPAVLOV_PATH}/requirements/tf.txt",
      "{DEEPPAVLOV_PATH}/requirements/gensim.txt",
      "{DEEPPAVLOV_PATH}/requirements/spacy.txt",
      "{DEEPPAVLOV_PATH}/requirements/en_core_web_sm.txt"
    ],
    "labels": {
      "telegram_utils": "GoalOrientedBot",
      "server_utils": "GoalOrientedBot"
    },
    "download": [
      {
        "url": "http://files.deeppavlov.ai/embeddings/glove.6B.100d.txt",
        "subdir": "{DOWNLOADS_PATH}/embeddings"
      }
    ]
  }
}

In [ ]:
from deeppavlov import configs, parse_json

